**This cell reads the sheets on google sheet and gives each column a uuid,if the same column name comes in different sheet then will get the same uuid
**

In [ ]:
import pandas as pd
import uuid
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials

# Dictionary to store UUIDs for each unique column name across sheets
uuid_mapping = {}

# Function to generate UUID for columns and apply to DataFrame
def apply_uuid_to_columns(df):
    new_columns = {}
    for col in df.columns:
        if col in uuid_mapping:
            new_columns[col] = f"{col}--{uuid_mapping[col]}"
        else:
            generated_uuid = str(uuid.uuid4())
            uuid_mapping[col] = generated_uuid
            new_columns[col] = f"{col}--{generated_uuid}"

    df.rename(columns=new_columns, inplace=True)
    return df

# Function to upload DataFrame to a specific sheet in a new Google Sheet
def upload_to_new_google_sheets(df, new_sheet_url, creds_json, specific_sheet_name=None):
    scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_json, scope)
    client = gspread.authorize(creds)

    # Open the new Google Sheet by its URL
    new_sheet = client.open_by_url(new_sheet_url)

    # Create the sheet if it doesn't exist
    try:
        worksheet = new_sheet.worksheet(specific_sheet_name)
    except gspread.exceptions.WorksheetNotFound:
        worksheet = new_sheet.add_worksheet(title=specific_sheet_name, rows="600", cols="600")

    # Clear the sheet before uploading new data (optional)
    worksheet.clear()

    # Write DataFrame to the new Google Sheet
    set_with_dataframe(worksheet, df)

    print(f"Data uploaded to {new_sheet_url}, sheet: {specific_sheet_name}")

# Main function to process all sheets from the input Google Sheet
def process_and_upload_all_sheets(input_sheet_url, new_sheet_url, creds_json):
    scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name(creds_json, scope)
    client = gspread.authorize(creds)

    # Open the input Google Sheet by its URL
    input_sheet = client.open_by_url(input_sheet_url)

    # Get all sheet names from the input Google Sheet
    sheet_names_list = [sheet.title for sheet in input_sheet.worksheets()]

    # Loop through each sheet
    for sheet_name in sheet_names_list:
        # Read the sheet as a DataFrame
        worksheet = input_sheet.worksheet(sheet_name)
        df = get_as_dataframe(worksheet, dtype=str)  # Convert all values to string to avoid issues

        # Apply UUIDs to columns
        df_with_uuid = apply_uuid_to_columns(df)

        # Upload to the new Google Sheet with the same sheet name
        upload_to_new_google_sheets(df_with_uuid, new_sheet_url, creds_json, specific_sheet_name=sheet_name)

# Example usage
creds_json = ''  # Path to your credentials JSON
input_sheet_url = ''  # Input Google Sheet URL
new_sheet_url = ''  # Output Google Sheet URL

# Process and upload all sheets from input to new Google Sheet
process_and_upload_all_sheets(input_sheet_url, new_sheet_url, creds_json)


*this cell gets all the unique responses in all the columns (if the responses are seperated by ';') and make a dataframe which contains columns like Question,Question UUID,	Responses,	Category,	Project ID.
*

In [ ]:
from google.colab import auth
auth.authenticate_user()

import pandas as pd

import gspread
from oauth2client.client import GoogleCredentials
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
wb = gc.open_by_key('1lIThn4TGgm-6SyQgVN9qYJeLr7BdsgtTGeot77XFdbI')
ws = wb.worksheet('Odisha')
rows = ws.get_all_values()
df = pd.DataFrame(rows[1:], columns=rows[0])

# Create a list of questions and their corresponding responses for 'Action'
questions_actions = [
    '2.4 What is the most viable action for addressing the challenges of heatwaves, particularly for health, mental well-being, and daily routines in your community?--3e4301bf-ecd9-4785-a152-286e7fea4c6e',
    '3.4 Which initiative should be prioritized to support poor communities in managing income loss, business operation disruptions, and employment instability?--7b079dee-2c59-40c8-9ea6-6ada85e14af7',
    '4.4 How can poor communities best mitigate the financial burden caused by heatwaves, including the costs associated with water, cooling, and healthcare?--67512f63-05ae-4589-af9b-63c8607113e9',
    '6.6 What specific demands are communities making for anticipatory measures to combat the risks associated with heat waves?--93fd070b-50d0-473e-8ddf-563fabba36f0',
    '7.1 Which of the following cash-based relief interventions do you believe would be most beneficial for victims of heatwaves?--4a60da82-0983-4b0a-b392-6b93af8958ba',
    '7.2 In the wake of a severe heatwave, what type of compensation should be offered to individuals and families affected by hospitalization, income loss, and death?--0dedcb9e-a396-4f06-89ba-742d033a74ea',
    '7.3 What low-cost financing mechanisms are most effective for supporting heatwave mitigation and adaptation efforts in vulnerable communities?--77498aad-1b17-4dfa-bb00-223e2857feaf',
    '7.4 What is the most appropriate way to adapt social security and protection systems to assist victims of heatwaves?--bc7a0673-0dd9-4f51-8a51-3a5e8ba73e80',
    '7.5 Which of the following is the most pressing need for individuals affected by heatwaves?--bf16755f-d99c-4b4e-8759-a066d3895365'
]

# Get responses for each action question
responses_actions = [
    df.get('2.4 What is the most viable action for addressing the challenges of heatwaves, particularly for health, mental well-being, and daily routines in your community?--3e4301bf-ecd9-4785-a152-286e7fea4c6e', '').astype(str),
    df.get('3.4 Which initiative should be prioritized to support poor communities in managing income loss, business operation disruptions, and employment instability?--7b079dee-2c59-40c8-9ea6-6ada85e14af7', '').astype(str),
    df.get('4.4 How can poor communities best mitigate the financial burden caused by heatwaves, including the costs associated with water, cooling, and healthcare?--67512f63-05ae-4589-af9b-63c8607113e9', '').astype(str),
    df.get('6.6 What specific demands are communities making for anticipatory measures to combat the risks associated with heat waves?--93fd070b-50d0-473e-8ddf-563fabba36f0', '').astype(str),
    df.get('7.1 Which of the following cash-based relief interventions do you believe would be most beneficial for victims of heatwaves?--4a60da82-0983-4b0a-b392-6b93af8958ba', '').astype(str),
    df.get('7.2 In the wake of a severe heatwave, what type of compensation should be offered to individuals and families affected by hospitalization, income loss, and death?--0dedcb9e-a396-4f06-89ba-742d033a74ea', '').astype(str),
    df.get('7.3 What low-cost financing mechanisms are most effective for supporting heatwave mitigation and adaptation efforts in vulnerable communities?--77498aad-1b17-4dfa-bb00-223e2857feaf', '').astype(str),
    df.get('7.4 What is the most appropriate way to adapt social security and protection systems to assist victims of heatwaves?--bc7a0673-0dd9-4f51-8a51-3a5e8ba73e80', '').astype(str),
    df.get('7.5 Which of the following is the most pressing need for individuals affected by heatwaves?--bf16755f-d99c-4b4e-8759-a066d3895365', '').astype(str)
]

# Create a list of questions and their corresponding responses for 'Measures Anticipated'
questions_measures_anticipated = [
    '6.2 What behavioral adjustments do you make in response to heatwave advisories?--20023e40-ac79-4c5b-91f2-3fdd47bb0a29',
    '6.3 How do you modify your home in response to heatwave advisories?--5798d0ef-5802-45e8-91ef-634449697b32',
    '6.5 Which of the following approaches have you considered for financing losses and damages caused by heatwaves?--959c3f83-7892-4e8c-9385-7566a6a6681b',
    '6.4 Since the last heatwave, what investments have you made to reduce the impact of future heatwaves on your household?--c54419b1-8d70-4afc-b23c-f6c732793b84'
]

# Get responses for each measures question
responses_measures_anticipated = [
    df.get('6.2 What behavioral adjustments do you make in response to heatwave advisories?--20023e40-ac79-4c5b-91f2-3fdd47bb0a29', '').astype(str),
    df.get('6.3 How do you modify your home in response to heatwave advisories?--5798d0ef-5802-45e8-91ef-634449697b32', '').astype(str),
    df.get('6.5 Which of the following approaches have you considered for financing losses and damages caused by heatwaves?--959c3f83-7892-4e8c-9385-7566a6a6681b', '').astype(str),
    df.get('6.4 Since the last heatwave, what investments have you made to reduce the impact of future heatwaves on your household?--c54419b1-8d70-4afc-b23c-f6c732793b84', '').astype(str)
]

# Function to create a DataFrame from questions and responses, including project ID
def create_df(questions, responses, category, df_main):
    question_list = []
    question_uuid_list = []
    unique_responses_list = []

    for i, res_series in enumerate(responses):
        full_question = questions[i]
        question, uuid = full_question.split('--')  # Split by '--' to separate question and UUID
        unique_responses = res_series.drop_duplicates().tolist()  # Get unique responses

        # Append to lists
        question_list.append(question.strip())
        question_uuid_list.append(uuid.strip())
        unique_responses_list.append(unique_responses)

    # Create DataFrame
    df_combined = pd.DataFrame({
        'Question': question_list,
        'Question UUID': question_uuid_list,
        'Responses': unique_responses_list,
        'Category': [category] * len(question_list),
        'Project ID': df_main['Project ID--02cc4a8c-43e9-4671-9dd7-c9602345fe49'].iloc[0]  # Project ID from the original df
    })
    return df_combined

# Create DataFrames for both categories
df_actions = create_df(questions_actions, responses_actions, 'Action', df)
df_measures = create_df(questions_measures_anticipated, responses_measures_anticipated, 'Measures Anticipated', df)

# Combine both DataFrames
df_Odisha = pd.concat([df_actions, df_measures], ignore_index=True)

# Display the final DataFrame
df_Odisha


# Function to extract unique values from a string column separated by ';'
def get_unique_values_from_column(series):
    all_values = []
    for row in series:
        if isinstance(row, str):  # Ensure it's a string before processing
            all_values.extend([val.strip() for val in row.split(';') if val.strip()])
    unique_values = list(set(all_values))  # Get unique responses across all rows
    return unique_values

# Function to create a DataFrame from questions and responses, including project ID
def create_df(questions, responses, category, df_main):
    question_list = []
    question_uuid_list = []
    unique_responses_list = []

    for i, res_series in enumerate(responses):
        full_question = questions[i]
        question, uuid = full_question.split('--')  # Split by '--' to separate question and UUID

        # Apply the unique value extraction function to each response column
        unique_responses = get_unique_values_from_column(res_series)

        # Append to lists
        question_list.append(question.strip())
        question_uuid_list.append(uuid.strip())
        unique_responses_list.append(unique_responses)

    # Create DataFrame
    df_combined = pd.DataFrame({
        'Question': question_list,
        'Question UUID': question_uuid_list,
        'Responses': unique_responses_list,
        'Category': [category] * len(question_list),
        'Project ID': df_main['Project ID--02cc4a8c-43e9-4671-9dd7-c9602345fe49'].iloc[0]  # Project ID from the original df
    })
    return df_combined



# Actions
df_actions = create_df(questions_actions, responses_actions, 'Action', df)

# Measures Anticipated
df_measures = create_df(questions_measures_anticipated, responses_measures_anticipated, 'Measures Anticipated', df)

# Measures Taken


# Combine all DataFrames
df_Odisha = pd.concat([df_actions, df_measures], ignore_index=True)

# Display the final DataFrame
df_Odisha


**If we have all the surveys like this then we can concate them together**

In [ ]:
master_df=pd.concat([df_Ahmedabad_Gujarat_1,df_Delhi,df_Gujarat_2,df_Patan_Gujarat,df_Varanasi,df_Maharashtra,df_Rajasthan, df_Odisha, df_Kerala, df_Uttar_pradesh, df_Lucknow_Uttar_pradesh,df_Gujarat],ignore_index=True)
master_df

**This cell cleans the survey by removing new line characters and seperating the question number from the question keeping it in seperate column**

In [ ]:
import re
import pandas as pd

# Function to clean and format unique responses
# Function to clean and format unique responses, including replacing line breaks with spaces
def clean_and_format_responses(responses):
    cleaned_responses = set([resp.replace('\n', ' ').strip() for resp in sum(responses.tolist(), [])])
    return [f'"{resp}"' for resp in cleaned_responses if resp]

# Rest of the code remains the same


# Function to extract the number from the beginning of a question
def extract_question_number(question):
    # Regex to match both numbers with decimals like 2.4 and numbers with trailing period like 20.
    match = re.match(r'^\d+\.\d+|^\d+\.', question)
    if match:
        return match.group(0).strip('.')  # Return the matched number, remove trailing dot if present
    return None

def remove_question_number(question):
    return re.sub(r'^\d+\.\d+|^\d+\.', '', question).strip()  # Remove the number and any trailing spaces

# Group by 'Question UUID' and aggregate 'Responses' and 'Project ID'
df_grouped = master_df.groupby(['Question UUID']).agg({
    'Question': 'first',  # Keep the first 'Question' for each UUID
    'Category': 'first',  # Keep the first 'Category' for each UUID
    'Project ID': lambda x: list(x.unique()),  # Collect unique 'Project ID's into a list
    'Responses': lambda x: clean_and_format_responses(x)  # Clean, remove duplicates, and format responses
}).reset_index()

# Add the 'Question Number' and clean 'Question' columns
df_grouped['Question Number'] = df_grouped['Question'].apply(extract_question_number)  # Extract number
df_grouped['Question'] = df_grouped['Question'].apply(remove_question_number)  # Remove number from question text

# Display the result
df_grouped

